In [1]:
import gc

import numpy as np
import pandas as pd
import spacy
import textacy
import utils
import utils_clean
import utils_text
from collections import Counter
from gensim.models import KeyedVectors
from keras.preprocessing import sequence, text
from nltk.corpus import stopwords
from tqdm import tqdm

/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/home/w/anaconda3/envs/idp3exp/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: insp

/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/w/anaconda3/envs/idp3exp/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
 

In [2]:
src = '../data/'

train, test = utils.load_data(src, mode='BasicClean')
data = pd.concat([train, test]).reset_index(drop=True)

Load data with basic cleaning.


In [7]:
train['len'] = train.comment_text.apply(lambda x: len(x))
train.len.describe()

count    159571.000000
mean        342.575593
std         516.371582
min           3.000000
25%          84.000000
50%         179.000000
75%         378.000000
max        7142.000000
Name: len, dtype: float64

In [8]:
test['len'] = test.comment_text.apply(lambda x: len(x))
test.len.describe()

count    153164.000000
mean        309.939699
std         511.835190
min           0.000000
25%          66.000000
50%         153.000000
75%         334.000000
max       11801.000000
Name: len, dtype: float64

In [3]:
def get_weight(count, eps=10000, min_count=2):
    return 0 if count < min_count else 1 / (count + eps)


def add_word_count(x, df, word):
    x['comment_' + word] = df['comment_text'].apply(lambda x: (word in str(x).lower())*1)

    
def try_apply_dict(x,dict_to_apply):
    try:
        return dict_to_apply[x]
    except KeyError:
        return 0

In [10]:
train_qs = pd.Series(data['comment_text'].tolist()).astype(str)
words = (" ".join(train_qs)).lower().split()
counts = Counter(words)
weights = {word: get_weight(count) for word, count in counts.items()}

In [12]:
x = pd.DataFrame()


x['len_q1'] = data['comment_text'].apply(lambda x: len(str(x)))
x['len_char_q1'] = data['comment_text'].apply(lambda x: len(str(x).replace(' ', '')))
x['len_word_q1'] = data['comment_text'].apply(lambda x: len(str(x).split()))
x['duplicated'] = data.duplicated(['comment_text']).astype(int)

add_word_count(x, data,'how')
add_word_count(x, data,'what')
add_word_count(x, data,'which')
add_word_count(x, data,'who')
add_word_count(x, data,'where')
add_word_count(x, data,'when')
add_word_count(x, data,'why')

add_word_count(x, data,'ass')
add_word_count(x, data,'tit')
add_word_count(x, data,'hell')
add_word_count(x, data,'cum')
add_word_count(x, data,'cipa')
add_word_count(x, data,'sex')
add_word_count(x, data,'butt')
add_word_count(x, data,'fuck')
add_word_count(x, data,'bitch')
add_word_count(x, data,'cock')
add_word_count(x, data,'fag')
add_word_count(x, data,'gay')
add_word_count(x, data,'nazi')

In [13]:
x['selected_badwords_sum'] = x.iloc[:, 11:-2].sum(axis=1)
x.to_pickle('../data/features/data_TextStatistics.pkl')

In [ ]:
x.selected_badwords_sum.value_counts()